Generate choicefile

In [1]:
trafficfilenamearr = ["unv1","prv1","cluster_a","cluster_b","cluster_c"]
numintervalarr = [4,4,24,24,24]
for itrafficfilename,trafficfilename in enumerate(trafficfilenamearr):
    numinterval = numintervalarr[itrafficfilename]
    choicefile = f"/home/annzhou/DRing/src/emp/datacentre/choicefiles/{trafficfilename}"
    with open(choicefile,'w') as f:
        for _ in range(numinterval):
            f.write("5\n")

Generate qivarfiles for meta
inspired by computerouting2/computerouting2.ipynb

In [4]:
def copy(copyfromfile, copyintofile):
    with open(copyfromfile,'r') as fr:
        lines = fr.readlines()
        with open(copyintofile,'w') as fw:
            for line in lines:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
                pid = int(tokens[2])
                value = float(tokens[5])
                fw.write(f"{fromsw},{tosw},{pid},{value}\n")

In [5]:
clusterarr = ["a","b","c"]
intervalstart = 24
intervalend = 48
factor = 64
numsw = 80
racke = 0
step = 1800

for cluster in clusterarr:
    for interval in range(intervalstart, intervalend):
        copyfromivarfile = f"/home/annzhou/DRing/src/emp/datacentre/pathweightfiles/dring/racke{racke}/cluster{cluster}/pathweight_dring_racke{racke}_cluster{cluster}_lp1_barriernocrossover_{interval*step}_{(interval+1)*step}_{factor}.txt"
        copyintoqivarfile = f"/home/annzhou/DRing/src/emp/datacentre/qivarfiles/qivar_dring_0_0_cluster_{cluster}_{interval}_racke{racke}_2_0"
        copy(copyfromivarfile, copyintoqivarfile)

Generate qivarfiles for wisc
inspired by computerouting2/computerouting2.ipynb

generate conffile

In [24]:
trafficname = "unv1"
multstrarr1 = ["0 2 25","0 17 100","0 1 4","0 17 50","0 21 50","0 51 100"]
# multstrarr2 = ["0 21 50","0 51 100","0 59 100","0 34 50","0 38 50","0 17 20"]
multnamearr1 = range(5,31,5)
# multnamearr2 = range(25,51,5)
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/routing_fhi/{trafficname}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 4 8 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 4 8 0 0 0 0 fhi 0 {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

In [26]:
trafficname = "prv1"
multstrarr1 = ["0 2 100","0 5 100","0 7 100","0 10 100","0 12 100","0 15 100","0 17 100","0 20 100","0 22 100","0 25 100"]
# multstrarr2 = ["0 22 100"]
multnamearr1 = range(2,21,2)
# multnamearr2 = [18]
# multstrarr1 = ["0 25 100","0 37 100","0 49 100","0 62 100","0 74 100"]
# multstrarr2 = []
# multnamearr1 = range(20,61,10)
# multnamearr2 = []
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/routing_fhi/{trafficname}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 4 8 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 4 8 0 0 0 0 fhi 0 {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

In [7]:
trafficname = "cluster_a"
racke=0
multstrarr1 = ["4 8 100","8 17 100","12 25 100","16 34 100","20 42 100","24 51 100","28 59 100","32 67 100","36 76 100"]
multnamearr1 = range(10,91,10)
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/routing_racke/{trafficname}_{racke}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 24 48 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname}_{racke} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 24 48 0 0 0 0 racke {racke} {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{racke}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

In [2]:
trafficname = "cluster_b"
racke=0
multstrarr1 = ["0 27 100","0 53 100","0 80 100","1 7 100","1 34 100","1 60 100","1 87 100","2 14 100","2 40 100"]
multnamearr1 = range(10,91,10)
sseedarr = range(1,6)

conffile = f"/u/az6922/DRing/src/emp/datacentre/experiments/routing_racke/{trafficname}_{racke}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 24 48 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname}_{racke} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 24 48 0 0 0 0 racke {racke} {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{racke}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

In [9]:
trafficname = "cluster_c"
racke=0
multstrarr1 = ["0 19 100","0 38 100","0 57 100","0 75 100","0 94 100","1 13 100","1 32 100","1 51 100","1 70 100"]
multnamearr1 = range(10,91,10)
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/routing_racke/{trafficname}_{racke}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 24 48 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname}_{racke} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 24 48 0 0 0 0 racke {racke} {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{racke}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

generate sumfile & picklefile

In [ ]:
import pickle

trafficname = "prv1"

homedir = "/home/annzhou"
datadict = dict()
conffile = f"{homedir}/DRing/src/emp/datacentre/experiments/routing_fhi/{trafficname}.conf"
sumfile = f"{homedir}/DRing/src/emp/datacentre/experiments/routing_fhi/{trafficname}.txt"
with open(conffile,'r') as f:
    with open(sumfile,'a') as sumf:
        lines = f.readlines()
        for line in lines:
            tokens = line.split()
            logfile = tokens[32]
            sizelist = list()
            durationlist = list()
            startlist = list()
            with open(f"{homedir}/DRing/src/emp/datacentre/{logfile}",'r') as logf:
                print(logfile)
                loglines = logf.readlines()
                for logline in loglines:
                    logtokens = logline.split()
                    if logtokens[0] == "FCT":
                        durationlist.append(float(logtokens[2]))
                        sizelist.append(int(logtokens[1]))
                        startlist.append(float(logtokens[3]))
            datadict[logfile] = [durationlist,sizelist,startlist]
            durationlist.sort()
            sumf.write(f"{logfile}\t{sum(durationlist)/len(durationlist)}\t{durationlist[int(len(durationlist)*0.5)]}\t{durationlist[int(len(durationlist)*0.99)]}\t{durationlist[int(len(durationlist)*0.9999)]}\n")

with open(f'{homedir}/DRing/src/emp/datacentre/experiments/routing_fhi/{trafficname}.pickle', 'wb') as handle:
    pickle.dump(datadict, handle, protocol=pickle.HIGHEST_PROTOCOL)

delete log,run files

In [ ]:
import os

homedir = "/home/annzhou"
dringdir = f"{homedir}/DRing/src/emp/datacentre"
for filedir in os.listdir(dringdir):
    if os.path.isfile(os.path.join(dringdir, filedir)):
        tokens = filedir.split("_")
        if tokens[0]=="m" and (tokens[1]=="leafspine" or tokens[1]=="dring") and tokens[-1].split('.')[-1]=="log":
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")
        if (tokens[0]=="rrg" or tokens[0]=="leafspine") and tokens[1]=="log":
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")
        if (tokens[-1]=="run") and ((tokens[0]=="leafspine" and tokens[1]=="leafspine") or (tokens[0]=="rrg" and (tokens[1]=="rrg" or tokens[1]=="dring"))):
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")